In [5]:
import sys
sys.path.append("/home/oscar/projects/ai-sales-agent")
from agents.config.llm_models import AVAILABLE_MODELS

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["user_input"],
    template="""
Eres un asistente que ayuda a normalizar las preferencias de autos dadas por un usuario.

Tu tarea es extraer y corregir la marca, el modelo y el año de un automóvil si están presentes, incluso si hay errores ortográficos.

Solo responde con un JSON con las claves: "marca", "modelo" y "año". Si algún dato no está claro, deja el campo como null.

Ejemplos:

Usuario: "kiero un chebrolet onix 2021 con menos de 50000 km"
Respuesta:
{{
  "marca": "Chevrolet",
  "modelo": "Onix",
  "año": 2021,
  "version": null,
  "precio": null,
  "km": 50000,
  "bluetooth": null,
  "carplay": null
}}

Usuario: "toyotaa corola 2018 con carplay "
Respuesta:
{{
  "marca": "Toyota",
  "modelo": "Corolla",
  "año": 2018,
  "version": null,
  "precio": null,
  "km": null,
  "bluetooth": null,
  "carplay": Sí
}}

Usuario: "un mazda porfa"
Respuesta:
{{
  "marca": "Mazda",
  "modelo": null,
  "año": null,
  "version": null,
  "precio": null,
  "km": null,
  "bluetooth": null,
  "carplay": null
}}

Ahora analiza este mensaje:
"{user_input}"
"""
)

model = "deepseek-v3"
model = "gpt-3.5-turbo"
model_info = AVAILABLE_MODELS[model]
llm = ChatOpenAI(
    model=model_info["model_name"],
    openai_api_key=model_info["api_key"],
    openai_api_base=model_info["url"],
    temperature=0    
)
normalize_chain = LLMChain(llm=llm, prompt=prompt)

# Ejemplo de uso
user_msg = "busco un hondaa sivic 2016"
output = normalize_chain.run(user_input=user_msg)
print(output)


{
  "marca": "Honda",
  "modelo": "Civic",
  "año": 2016,
  "version": null,
  "precio": null,
  "km": null,
  "bluetooth": null,
  "carplay": null
}


In [ ]:
import pandas as pd

df = pd.read_csv("/home/oscar/projects/ai-sales-agent/data/csv/sample_caso_ai_engineer.csv")
def clasificar_auto_general(largo_cm, ancho_cm, alto_cm):
    resultado = {}
    largo_m = largo_cm / 100
    ancho_m = ancho_cm / 100
    alto_m = alto_cm / 100
    # Clasificación por tamaño
    if largo_m < 3.7:
        resultado["tamaño"] = "Auto pequeño / urbano"
    elif 3.7 <= largo_m < 4.1:
        resultado["tamaño"] = "Auto compacto"
    elif 4.1 <= largo_m < 4.5:
        resultado["tamaño"] = "Auto mediano"
    elif 4.5 <= largo_m < 4.85:
        resultado["tamaño"] = "Auto grande / familiar"
    else:
        resultado["tamaño"] = "Vehículo muy grande / ejecutivo o de carga"

    # Estimar tipo de carrocería
    if alto_m >= 1.65 and ancho_m >= 1.75:
        if largo_m > 4.8:
            resultado["carrocería"] = "SUV grande o camioneta"
        elif largo_m > 4.3:
            resultado["carrocería"] = "SUV mediano o crossover"
        else:
            resultado["carrocería"] = "Mini SUV / crossover compacto"
    elif alto_m < 1.45:
        if largo_m > 4.2 and ancho_m > 1.75:
            resultado["carrocería"] = "Coupé o sedán deportivo"
        else:
            resultado["carrocería"] = "Hatchback o compacto bajo"
    else:
        resultado["carrocería"] = "Sedán o auto tradicional"

    # Estimar uso típico
    if resultado["carrocería"] in ["SUV grande o camioneta", "SUV mediano o crossover"]:
        resultado["uso_típico"] = "Familiar, caminos difíciles o viajes largos"
    elif resultado["carrocería"] in ["Hatchback o compacto bajo", "Auto pequeño / urbano"]:
        resultado["uso_típico"] = "Uso urbano, económico y fácil de estacionar"
    elif "sedán" in resultado["carrocería"].lower():
        resultado["uso_típico"] = "Uso mixto: ciudad y carretera"
    elif "coupé" in resultado["carrocería"].lower():
        resultado["uso_típico"] = "Deportivo o recreativo"
    else:
        resultado["uso_típico"] = "Versátil / depende del equipamiento"

    return resultado



In [ ]:
df[""

,stock_id,km,precio,marca,modelo,año,version,bluetooth,largo,ancho,altura,car_play
0,243587,77400,461999.0,Volkswagen,Touareg,2018,3.0 V6 TDI WOLFSBURG EDITION AUTO 4WD,Sí,4801.0,1940.0,1709.0,NaN
1,229702,102184,660999.0,Land Rover,Discovery Sport,2018,2.0 HSE LUXURY AUTO 4WD,Sí,4599.0,2069.0,1724.0,NaN
2,160422,56419,866999.0,BMW,Serie 2,2018,3.0 M2 DCT,Sí,4468.0,1854.0,1410.0,Sí
3,308634,76000,238999.0,Toyota,Avanza,2018,1.5 XLE AT,Sí,4140.0,1660.0,1695.0,NaN
4,305016,29377,313999.0,Toyota,Corolla,2020,1.8 LE AUTO,Sí,4650.0,1776.0,1475.0,Sí


In [ ]:
df_resultado = df.apply(lambda row: pd.Series(clasificar_auto_general(row['largo_m'], row['ancho_m'], row['alto_m'])), axis=1)


In [ ]:
import pandas as pd
from rapidfuzz import fuzz, process
import json


weights = {
    "marca": 50,
    "modelo":  50,
    "año": 10,
    "version": 20,
    "precio": 30,
    "km": 20,
    "bluetooth": 20,
    "carplay": 20,
}

def normalize_weights(weights):
    total = sum(weights.values())
    for key in weights:
        weights[key] /= total
    return weights

def fuzzy_filter(df, user_input, weights=weights, km_tolerance=0.2):
    df = df.copy()
    weights = normalize_weights(weights)
    # Calcular score por campo
    for key in weights:
        val = user_input.get(key)
        if val:
            df[f"{key}_score"] = df[key].apply(lambda x: fuzz.ratio(str(val), str(x)))
        else:
            df[f"{key}_score"] = 10

    # Filtrado exacto o numérico
    for key, val in user_input.items():
        if val is None or key in weights:
            continue
        if key == "km":
            # min_km = val * (1 - km_tolerance)
            max_km = val * (1 + km_tolerance)
            df = df[df["km"].between(0, max_km)]
        else:
            df = df[df[key] == val]

    # Ponderación
    df["similarity_score"] = sum(
        df[f"{k}_score"] * w for k, w in weights.items()
    ) / sum(weights.values())

    df = df.sort_values(by="similarity_score", ascending=False)

    return df.drop(columns=[f"{k}_score" for k in weights]).reset_index(drop=True)

In [45]:
user_input = json.loads(output)
print(user_input)
resultados = fuzzy_filter(df, user_input)
resultados

{'marca': 'Honda', 'modelo': 'Civic', 'año': 2016, 'version': None, 'precio': None, 'km': None, 'bluetooth': None, 'carplay': None}
Score marca: 100.0
Score modelo: 100.0
Score año: 100.0
Score version: 10
Score precio: 10
Score km: 10
Score bluetooth: 10
Score carplay: 10


,stock_id,km,precio,marca,modelo,año,version,bluetooth,largo,ancho,altura,car_play,similarity_score
0,299048,64638,192999.0,Honda,Civic,2013,1.8 EX-L AT 4DRS,Sí,4555.0,1755.0,1450.0,NaN,53.863636
1,323039,67004,286999.0,Honda,CR-V,2016,2.4 I-STYLE,Sí,4575.0,1820.0,1655.0,NaN,37.323232
2,227162,74064,414999.0,Honda,CR-V,2017,2.4 EX,Sí,4586.0,1855.0,1657.0,NaN,36.186869
3,125904,95300,253999.0,Honda,Odyssey,2016,3.5 LX,Sí,5180.0,2010.0,1735.0,NaN,32.272727
4,242862,83100,314999.0,Honda,Odyssey,2015,3.5 EX AT,Sí,5180.0,2010.0,1735.0,NaN,31.136364
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,317264,100072,265999.0,KIA,Sportage,2018,2.0 LX AUTO,Sí,4480.0,1855.0,1645.0,Sí,8.409091
96,290164,12279,275999.0,MG,MG5,2022,1.5 ELEGANCE CVT,Sí,4601.0,1818.0,1489.0,Sí,7.272727
97,312204,39492,331999.0,KIA,FORTE,2020,1.6 GT DCT,Sí,4640.0,1800.0,1450.0,Sí,7.272727
98,322962,21000,408999.0,KIA,FORTE,2023,2.0 GT LINE IVT,Sí,4640.0,1800.0,1450.0,Sí,7.272727


In [22]:
user_input

{'marca': 'Honda',
 'modelo': 'Civic',
 'año': 2020,
 'version': None,
 'precio': None,
 'km': None,
 'bluetooth': None,
 'carplay': None}